# Data Prepration

## how to define holiday

any day in which consecusitively there are zero sales for more than 12 hours can be treated as store closed we have to augment the data for this as given data is for the number of days and then we have converted days into hours so we can just randomly fill every 7th day as store closed and can mark all the sales to zero and we can select some random days also to flag the as zero

## how to define promotion day

sales greater than 90th percentile can be treated as promotion days we can see first the distribution of data that what are the unit sales value for 95th percentile of the data and if there are more number of days then we can push this to 97 percentile

In [54]:
import os
import pandas as pd
import numpy as np

In [2]:
# abs_data_path = r"D:\ds_projects\kaggle\m5\data"
abs_data_path = r"C:\Users\rahulgup\Downloads\m5-forecasting-accuracy"

In [41]:
# products available in store shelf to realtime analysis
max_prods = 7

In [3]:
sell_price_path = os.path.join(abs_data_path, "sell_prices.csv") 
sales_data_path = os.path.join(abs_data_path, "sales_train_validation.csv") 
calendar_data_path = os.path.join(abs_data_path, "calendar.csv")

In [4]:
calendar = pd.read_csv(calendar_data_path, keep_default_na=False)
sales_train_validation = pd.read_csv(sales_data_path, keep_default_na=False)
# sample_submission = pd.read_csv("sample_submission.csv", keep_default_na=False)
# sell_prices = pd.read_csv(sell_price_path, keep_default_na=False)

In [5]:
# sales_train_validation.describe()

In [6]:
# only one store is available for analysis at realtime
store = sales_train_validation.store_id.unique()[0]
store

'CA_1'

In [7]:
x = list(sales_train_validation.columns[6:])
change_col = [
    [
        "d_{}".format(x//12+1), #date_id
        x//12+1, #dt
        x%12+8 #hour
#         (x/12)%7+1 #day
    ] for x in range(len(x))
]
change_col[:5], change_col[-1:-5:-1],# change_col

([['d_1', 1, 8],
  ['d_1', 1, 9],
  ['d_1', 1, 10],
  ['d_1', 1, 11],
  ['d_1', 1, 12]],
 [['d_160', 160, 12],
  ['d_160', 160, 11],
  ['d_160', 160, 10],
  ['d_160', 160, 9]])

In [8]:
date_df = pd.DataFrame(
    change_col, columns=[
        "date_id", "dt", "hour"
        #, "day"
    ]
)

In [9]:
sales_train_validation = sales_train_validation[sales_train_validation.store_id == store]

In [10]:
sales_train = pd.melt(
    sales_train_validation,
    id_vars=sales_train_validation.columns[:6],
    value_vars=sales_train_validation.columns[6:],
    var_name='date_id',
    value_name='quantity'
).merge(
    date_df,
    on="date_id"
)

In [11]:
sales_train = sales_train[
    list(sales_train_validation.columns[:6])+list(date_df.columns)+["quantity"]
][sales_train.quantity > 0]

In [12]:
len(sales_train)

1452095

In [13]:
sales_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,dt,hour,quantity
84,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,8,12
85,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,9,12
86,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,10,12
87,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,11,12
88,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,12,12


In [14]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,,,,,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,,,,,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,,,,,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,,,,,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,,,,,1,0,1


In [15]:
num_days = date_df.dt.max()
last_index = int(calendar[calendar.wday == num_days % 7].reset_index()[["index"]].max()[0])

In [16]:
last_index-num_days, last_index

(1805, 1965)

In [17]:
calendar_train = calendar[
    (calendar.index > (last_index - num_days))
    & (calendar.index <= last_index)
].reset_index()
calendar_train["date_id"] = "d_" + (calendar_train["index"] - (last_index - num_days)).map(str)
calendar_train = calendar_train.drop(
    ["index", "d", "wm_yr_wk", 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI'],
    axis=1
)

In [18]:
calendar_train.head()

,date,weekday,wday,month,year,date_id
0,2016-01-09,Saturday,1,1,2016,d_1
1,2016-01-10,Sunday,2,1,2016,d_2
2,2016-01-11,Monday,3,1,2016,d_3
3,2016-01-12,Tuesday,4,1,2016,d_4
4,2016-01-13,Wednesday,5,1,2016,d_5


In [19]:
# sales_train.to_csv(os.path.join(abs_data_path, "sales_train.csv"), index=False)

In [20]:
# calendar.to_csv(os.path.join(abs_data_path, "calendar_train.csv"), index=False)

---

# logic

## for store closure 
we can use the logic that any store showing continous 0 sales for more than we can flag them as store being closed

## to identify the run rate for the particular product 

we can run the univariate for first store product combination at row level and then at product level for the same day and  can see the results this will give us the matrix of 12 hours * 7 week days combination 

run rate will be useful measure to trigger the msg if the shelf is getting emptied 

# Core Logic

In [21]:
sales_train_final = sales_train.merge(
    calendar_train,
    on="date_id"
)

In [22]:
sales_train_final.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,dt,hour,quantity,date,weekday,wday,month,year
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,8,12,2016-01-09,Saturday,1,1,2016
1,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,9,12,2016-01-09,Saturday,1,1,2016
2,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,10,12,2016-01-09,Saturday,1,1,2016
3,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,11,12,2016-01-09,Saturday,1,1,2016
4,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,12,12,2016-01-09,Saturday,1,1,2016


In [23]:
sales_train_final.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date_id',
       'dt', 'hour', 'quantity', 'date', 'weekday', 'wday', 'month', 'year'],
      dtype='object')

In [24]:
# sales_train_final.to_csv(os.path.join(abs_data_path, "sales_cal_train.csv"), index=False)

# Tasks
- 

In [25]:
sales_threshold = sales_train_final.groupby(["id", "wday", "hour"])[["quantity"]].quantile(q=.25)
sales_threshold["threshold"] = sales_threshold["quantity"]
sales_threshold = sales_threshold.drop("quantity", axis=1)
sales_threshold.head()

0.25                                   threshold
id                          wday hour           
FOODS_1_001_CA_1_validation 1    8           1.0
                                 9           1.0
                                 10          1.0
                                 11          1.0
                                 12          1.0

In [26]:
new_sales = sales_train_final.merge(
    sales_threshold,
    on=["id", "wday", "hour"]
)
new_sales = new_sales[new_sales.quantity > new_sales.threshold]
new_sales = new_sales.drop("threshold", axis=1)
new_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,dt,hour,quantity,date,weekday,wday,month,year
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,8,12,2016-01-09,Saturday,1,1,2016
3,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_22,22,8,10,2016-01-30,Saturday,1,1,2016
4,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_43,43,8,12,2016-02-20,Saturday,1,2,2016
5,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_85,85,8,16,2016-04-02,Saturday,1,4,2016
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_120,120,8,13,2016-05-07,Saturday,1,5,2016


In [27]:
len(new_sales)

758386

In [38]:
top_prods = new_sales.groupby(["id"])[["quantity"]].mean().sort_values("quantity", ascending=False).reset_index()
len(top_prods)

1563

In [46]:
n_top = top_prods[top_prods.index < max_prods][["id"]]
n_top["mapping"] = "p_" + (n_top.index + 1).map(str)
n_top = n_top.set_index("id")
n_top

,mapping
id,
FOODS_3_318_CA_1_validation,p_1
FOODS_3_090_CA_1_validation,p_2
FOODS_3_030_CA_1_validation,p_3
FOODS_3_587_CA_1_validation,p_4
FOODS_3_383_CA_1_validation,p_5
FOODS_3_219_CA_1_validation,p_6
FOODS_3_586_CA_1_validation,p_7


In [61]:
top_n_data = new_sales.join(
    n_top,
    on="id",
    how="inner"
)
top_n_data["hr"] = "h_" + top_n_data["hour"].map(str)
top_n_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,dt,hour,quantity,date,weekday,wday,month,year,mapping,hr
99901,FOODS_3_030_CA_1_validation,FOODS_3_030,FOODS_3,FOODS,CA_1,CA,d_8,8,8,195,2016-01-16,Saturday,1,1,2016,p_3,h_8
99903,FOODS_3_030_CA_1_validation,FOODS_3_030,FOODS_3,FOODS,CA_1,CA,d_8,8,9,195,2016-01-16,Saturday,1,1,2016,p_3,h_9
99905,FOODS_3_030_CA_1_validation,FOODS_3_030,FOODS_3,FOODS,CA_1,CA,d_8,8,10,195,2016-01-16,Saturday,1,1,2016,p_3,h_10
99907,FOODS_3_030_CA_1_validation,FOODS_3_030,FOODS_3,FOODS,CA_1,CA,d_8,8,11,195,2016-01-16,Saturday,1,1,2016,p_3,h_11
99909,FOODS_3_030_CA_1_validation,FOODS_3_030,FOODS_3,FOODS,CA_1,CA,d_8,8,12,195,2016-01-16,Saturday,1,1,2016,p_3,h_12


In [64]:
run_rate_table = pd.pivot_table(
    top_n_data, values="quantity", index=["mapping", "store_id", "wday"],
    columns=["hour"], aggfunc=np.median
).reset_index()
run_rate_table.head()

hour,mapping,store_id,wday,8,9,10,11,12,13,14,15,16,17,18,19
0,p_1,CA_1,1,204.0,204.0,204.0,204.0,204.0,204.0,204.0,204.0,204.0,204.0,204.0,204.0
1,p_1,CA_1,2,214.0,214.0,214.0,214.0,214.0,214.0,214.0,214.0,214.0,214.0,214.0,214.0
2,p_1,CA_1,3,94.0,94.0,94.0,94.0,94.0,94.0,94.0,94.0,94.0,94.0,94.0,94.0
3,p_1,CA_1,4,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0
4,p_1,CA_1,5,115.0,115.0,115.0,115.0,115.0,115.0,115.0,115.0,115.0,115.0,115.0,115.0


In [65]:
run_rate_path = os.path.join(abs_data_path, "run_rate_table.csv") 
run_rate_table.to_csv(run_rate_path, index=False)

In [3]:
# import dask.dataframe as dd
# import numpy as np
# import dask.array as da

# sell_price_new_path = os.path.join(abs_data_path, "sales_cal_train.csv") 
# sales_train_final = dd.read_csv(sell_price_new_path, keep_default_na=False)

# x = sales_train_final.groupby(
#     ["store_id", "wday"]
# )[["quantity"]]
# y = sales_train_final.groupby(
#     ["item_id", "store_id", "wday"]
# )[["quantity"]]

# # def aggr(s):
# #     s['mean'] = np.mean(s)
# #     s['std'] = np.std(s)
# #     s['median'] = np.percentile(s, q=50)
# #     s['75%'] = np.percentile(s, q=75)
# #     s['85%'] = np.percentile(s, q=85)
# #     s['95%'] = np.percentile(s, q=95)
# #     return s

# def aggr(s):
#     s['mean'] = s.mean()
#     s['std'] = s.std()
#     s['median'] = s.quantile(q=.50)
#     s['75%'] = s.quantile(q=.75)
#     s['85%'] = s.quantile(q=.85)
#     s['95%'] = s.quantile(q=.95)
#     return s

# x.apply(
#     aggr
# ).compute()

# x.apply(
#     lambda x: [np.mean(x), np.std(x), np.percentile(x, q=50), np.percentile(x, q=75), np.percentile(x, q=85), np.percentile(x, q=95)]
# ).compute()

# y.apply(
#     lambda x: [np.mean(x), np.std(x), np.percentile(x, q=50), np.percentile(x, q=75), np.percentile(x, q=85), np.percentile(x, q=95)]
# ).compute()

# # x.agg(
# #     [
# #         "mean", "std",
# # #         lambda x: np.percentile(x, q=25),
# # #         lambda x: np.percentile(x, q=50),
# # #         lambda x: np.percentile(x, q=75),
# # #         lambda x: np.percentile(x, q=85),
# # #         percentile(0.95)
# #     ]
# # ).compute()

# # import dask
# # A = dask.array.arange(100, chunks=3)
# # df = dask.dataframe.from_dask_array(A)
# # df.describe().compute()

# store closure spark code

In [ ]:
# def flag_n_continuous(
#         dataframe,
#         groupings,
#         identifier,
#         threshold=2,
#         filters=None,
#         flag=1,
#         flag_name="flag",
#         second_identifier=False,
#         second_identifier_name="identifier_flag"
# ):
#     """
#     Identify N continuous rows and flag with flag value
#     """
#     # find minimum value of identifier column
#     input_columns = dataframe.columns
#     if filters:
#         dataframe = dataframe.filter(
#             filters
#         )
#     dataframe = dataframe.withColumn(
#         "min_identifier",
#         getattr(F, "min")(identifier).over(
#             W.Window.partitionBy(
#                 groupings
#             )
#         )
#     ).withColumn(
#         "diff_identifier",
#         getattr(F, "col")(identifier) - getattr(F, "col")("min_identifier") + 1
#     ).withColumn(
#         "rank",
#         getattr(F, "dense_rank")().over(
#             W.Window.partitionBy(
#                 groupings
#             ).orderBy(
#                 identifier
#             )
#         )
#     ).withColumn(
#         second_identifier_name,
#         getattr(F, "col")("diff_identifier") - getattr(F, "col")("rank")
#     ).withColumn(
#         "count",
#         getattr(F, "count")(second_identifier_name).over(
#             W.Window.partitionBy(
#                 groupings + [second_identifier_name]
#             )
#         )
#     ).withColumn(
#         flag_name,
#         F.when(
#             getattr(F, "col")("count") >= threshold,
#             F.lit(flag)
#         ).otherwise(
#             F.lit(0)
#         )
#     )
#     input_columns.append(flag_name)
#     if second_identifier:
#         input_columns.append(second_identifier_name)
#     return dataframe.select(*input_columns)


# for inventory management
